In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import re
import os
from tqdm.notebook import tqdm

In [18]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis")
sentiment_toxicity = pipeline("sentiment-analysis", model="unitary/toxic-bert")

tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [6]:
def data_processing(sub, field = "submission", info=False):
    file_name = "data/" + field + "_" + sub + ".csv"
    df = pd.read_csv(file_name,index_col=0)
    if info:
        print("subreddit : " + sub)
        print(df.describe())
        print(df.info())
    return df

In [22]:
def toxicity_analysis(text_list):
    toxicity = []
    for comment in tqdm(pol_df_text):
        results = sentiment_toxicity(comment, **tokenizer_kwargs)
        toxicity.append(results[0])
    return toxicity

In [25]:
def dataframe_toixicity(df, column='body'):
    df_text = df[column].to_list()
    toxicity_score = toxicity_analysis(df_text)
    df_toxicity = pd.DataFrame(toxicity_score)
    df_toxicity.rename(columns = {'score':'toxicity_score'}, inplace = True)
    df= df.join(df_toxicity)
    return df

In [7]:
pol_df = data_processing("politics", field = "comment")

In [26]:
df = dataframe_toixicity(pol_df)

  0%|          | 0/4999 [00:00<?, ?it/s]

In [27]:
df.head()

,id,author,body,parent_id,score,subreddit,subreddit_id,submission_id,label,toxicity_score
0,hynym3j,AutoModerator,"\nAs a reminder, this subreddit [is for civil ...",t3_t2suj2,1,politics,t5_2cneq,t2suj2,toxic,0.000705
0,hyda5f5,AutoModerator,"\nAs a reminder, this subreddit [is for civil ...",t3_t11uq0,1,politics,t5_2cneq,t11uq0,toxic,0.000705
0,hrjyvwr,AutoModerator,"\nAs a reminder, this subreddit [is for civil ...",t3_rxqvwi,1,politics,t5_2cneq,rxqvwi,toxic,0.000705
0,idtblfd,AutoModerator,"\nAs a reminder, this subreddit [is for civil ...",t3_vl6xm2,1,politics,t5_2cneq,vl6xm2,toxic,0.000705
0,hx7l2y4,AutoModerator,"\nAs a reminder, this subreddit [is for civil ...",t3_su3zob,1,politics,t5_2cneq,su3zob,toxic,0.000705


In [31]:
df.sort_values(['toxicity_score'], ascending=False).head()

,id,author,body,parent_id,score,subreddit,subreddit_id,submission_id,label,toxicity_score
388,hrkqpch,anarcho-onychophora,Are there any historical polls of this questio...,t3_rxqvwi,1,politics,t5_2cneq,rxqvwi,toxic,0.998678
388,hye5ms2,nta1646,I’ve been noticing a lot of Pro-Trump people s...,t3_t11uq0,2,politics,t5_2cneq,t11uq0,toxic,0.998678
388,hyptqjf,quantifical,"Putin supporters, go fuck yourselves",t3_t2suj2,2,politics,t5_2cneq,t2suj2,toxic,0.998678
388,idtqk0r,D-F-B-81,Just imagine the roles reversed... \n\nHad the...,t3_vl6xm2,-2,politics,t5_2cneq,vl6xm2,toxic,0.998678
647,idu3vje,kryppla,Agree and why would this be an extreme opinion...,t3_vl6xm2,1,politics,t5_2cneq,vl6xm2,toxic,0.998590
